# Streaming Videos Cache Optimization Problem
Using cache servers, we can optimize requests for videos from a data center to endpoints. Based on the predicted requests from endpoints, can we find a way to optimize the distribution and storage of said videos?

## Input and Parsing
Data is provided as text. We can parse said data into various tokens. We read from `data/input.txt` in this case.
The `problem_description` array holds, in order, from 0 to 4, the number of videos, number of endpoints, number of request descriptions, number of cache servers, and the capacity of each cache server in megabytes.
The `video_size` array holds the size of each video in MB.
We then parse based on the ammount of endpoints, to connect each endpoint to the caches. The `endpoint_data_description` describes the latency between an endpoint (serves as the index of the array) and the data center (latency is the value stored), and the `endpoint_cache_description` has the key/value specification of key:(endpoint, cache) -> value:latency.
Finally, `request_description` is a dictionary that holds the ammount of requests a certain video at an endpoint holds, specification of key:(endpoint, video) -> value:nº of requests.

In [21]:
def parse_results(file: str):
    problem_description = []
    video_size = []
    endpoint_data_description = []
    endpoint_cache_description = {}
    request_description = {}
    with open('data/' + file, 'r') as file:
        line = file.readline()
        tokens = line.strip().split()
        for token in tokens:
            problem_description.append(int(token))
        line = file.readline()
        tokens = line.strip().split()
        for token in tokens:
            video_size.append(int(token))
        i = 0
        while i != problem_description[1]:
            line = file.readline()
            tokens = line.strip().split()
            endpoint_data_description.append(int(tokens[0]))
            connections = int(tokens[1])
            j = 0
            while j < connections:
                line = file.readline()
                tokens = line.strip().split()
                endpoint_cache_description[(i, tokens[0])] = tokens[1]
                j += 1
            i += 1
        i = 0
        c = 0
        while i != problem_description[2]:
            i+=1
            line = file.readline()
            tokens = line.strip().split()
            key = (tokens[1], tokens[0])
            if key in request_description:
                request_description[key] += tokens[2]
            else:
                request_description[key] = tokens[2]
    return problem_description, video_size, endpoint_data_description, endpoint_cache_description, request_description

problem_description, video_size, endpoint_data_description, endpoint_cache_description, request_description = parse_results('me_at_the_zoo.in')

## Problem State
The problem state is defined by a dictionary that maps a cache to a list of videos. We must careful with the underlying constraints of the total video sizes not surpassing cache size.

## Goal and Scoring
Our goal is to maximize time saved by the caches, for this, we must go through our current cache configuration and figure out how much time we are saving in total based on the requests. Then, we multiply this value, in milliseconds, by 1000, to get the score. 
Time Saved (Request Description) = Nº of Requests * min(Latency of Data Center - Latency of Cache with Video)
Also, when re-scoring our problem, it makes more sense to update the current score with the alteration instead of re-calculating the score from scratch.
The score presumes a valid problem state.

In [22]:
import math
def score(problem_state: dict, endpoint_data_description: list, endpoint_cache_description: dict, request_description: dict) -> float:
    score = 0
    for (endpoint, video), request_number in request_description:
        data_center_latency = endpoint_data_description[endpoint]
        cache_latency = data_center_latency
        for cache, videos in problem_state:
            if video in videos:
                cache_latency = min(cache_latency, endpoint_cache_description[(endpoint, cache)])
        score += (data_center_latency - cache_latency) * request_number
    return math.floor(score * 1000)

### Updating the score
Now, for computational efficiency effects, we create a re-score function that based on a cache change, a current score and the descriptions, updates the score.

In [ ]:
def re_score(problem_state):
    return

## Greedy Algorithm

In order to have an initial solution to which we can apply the several heuristics to, we will develop a **greedy algorithm** to calculate a **potential solution state**. Our goal is to use this as a **starting point** that already has some value and logic and that will hopefully lead us to a **nearly ideal end result**.

For our algorithm we need a way to **evaluate the quality or benefit** of a specific solution. With this intent in mind, we will compute a **simplified score** for each cache/video pair, that corresponds to the saved latency.

(DC latency - Cache latency) * no. of requests

 After saving the score for the pairs we will **sort them** and then treat this as a sort of **knapsack problem** - put them in caches in order of most to least time saved until all cache/video pairs are iterated over or until all caches are full.

 The result of this method will be a dictionary data structure where the keys are cache IDs and the values are the videos stored in them (as described in the problem state above)

 We also implemented two helper functions:

 ### get_saved_time(element, video)

 Calculates how much time is saved on requests, at a specific endpoint, if a specific video is stored on x cache.

 This value is then incremented to the score associated to the (cache, video) key if video was requested previously on another endpoint connected to cache or a new key,value pair is introduced in the dictionary.

 ### current_cap(cache, solution)

 Calculates the current occupied capacity of a certain cache given a problem state.

 This is helpful when trying to figure out if a video can still be cached.

In [52]:
def greedy_start():
    # empty solution dictionary
    solution = {cache: [] for cache in range(problem_description[3])}
    cache_cap = problem_description[4]
    scores = {}

    # calculate scores 
    for (endpoint, video) in request_description.keys():
        for element in endpoint_cache_description.keys():
            if int(element[0]) == int(endpoint):
                cache = element[1]
                saved_time = get_saved_time(element, video)
                #if exists
                if (cache, video) in scores:
                    scores[(cache, video)] += saved_time
                #if not
                else:
                    scores[(cache, video)] = saved_time
    
    scores = [(cache, video, saved_time) for (cache, video), saved_time in scores.items()]


    # sort (video, cache, cost) by cost (desc)
    scores.sort(key=lambda a: a[2], reverse=True)

    # iterate through scores and fill caches
    for (cache, video, sc) in scores:
        curr_cap = current_cap(cache, solution)
        if (curr_cap < cache_cap) and (video_size[int(video)] + curr_cap <= cache_cap) and (video not in solution[int(cache)]):
            solution[int(cache)].append(video)

    print("solution:\n")
    print(solution)
   
    # return greedy solution
    return solution

def get_saved_time(element, video):
    (endpoint, cache) = element

    data_center_latency = endpoint_data_description[endpoint]
    cache_latency = endpoint_cache_description[element]

    request_number = request_description.get((f"{endpoint}", f"{video}"), 0)

    return (int(data_center_latency) - int(cache_latency)) * int(request_number)

def current_cap(cache, solution):
    return sum(video_size[int(v)] for v in solution[int(cache)])

greedy_start()

solution:

{0: ['0', '8', '1', '65', '99', '5', '16', '10'], 1: ['0', '1', '13', '10', '7', '99', '16'], 2: ['0', '8', '4', '1', '16', '65', '99'], 3: ['0', '4', '2', '16', '5', '27', '10'], 4: ['0', '1', '13', '7', '10', '81', '16'], 5: ['0', '4', '2', '1', '10', '16', '81', '5'], 6: ['0', '4', '2', '1', '16', '5', '10'], 7: ['0', '1', '2', '4', '74', '16'], 8: ['0', '8', '1', '3', '16'], 9: ['0', '4', '8', '16', '1', '5', '82', '10']}


{0: ['0', '8', '1', '65', '99', '5', '16', '10'],
 1: ['0', '1', '13', '10', '7', '99', '16'],
 2: ['0', '8', '4', '1', '16', '65', '99'],
 3: ['0', '4', '2', '16', '5', '27', '10'],
 4: ['0', '1', '13', '7', '10', '81', '16'],
 5: ['0', '4', '2', '1', '10', '16', '81', '5'],
 6: ['0', '4', '2', '1', '16', '5', '10'],
 7: ['0', '1', '2', '4', '74', '16'],
 8: ['0', '8', '1', '3', '16'],
 9: ['0', '4', '8', '16', '1', '5', '82', '10']}